In [167]:
from bokeh.models import Legend
from bokeh.layouts import layout
from bokeh.palettes import Spectral6, OrRd3, GnBu3
from bokeh.transform import linear_cmap
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, BoxSelectTool, BoxZoomTool, UndoTool, ResetTool, PanTool, WheelZoomTool
from bokeh.io import output_notebook, output_file
from bokeh.models import ColumnDataSource, CustomJS, Slider, Range, Label, LabelSet, Range1d, Select
from bokeh.models import Div, RangeSlider, Spinner, MultiChoice

In [146]:
x = np.random.random(10)

In [148]:
y = pow(x,2)

In [152]:
df = pd.DataFrame([x,y], index=['x','y']).T
df['color'] = 'red'

In [166]:
df

,x,y,color
0,0.719997,0.518396,red
1,0.028687,0.000823,red
2,0.688399,0.473893,red
3,0.751216,0.564326,red
4,0.900936,0.811685,red
5,0.537086,0.288461,red
6,0.203608,0.041456,red
7,0.083769,0.007017,red
8,0.556654,0.309864,red
9,0.201177,0.040472,red


In [161]:
ColumnDataSource(df).column_names

['index', 'x', 'y', 'color']

In [168]:
ColumnDataSource(df),title='x', value='0.900936')

TypeError: select() got an unexpected keyword argument 'title'

In [155]:
output_notebook()
p = figure()
p.circle(x='x', y='y', color='color', source=ColumnDataSource(df))
show(p)

Loading BokehJS ...

In [121]:
# from bokeh.models import MultiChoice


def plot_chr(df_chr):
#     output_notebook()
    output_file('test_out.html')
    df = df_chr.copy()
    df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce')
    df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
    df = df.sort_values(by=['chr_int', 'Position'], ascending=True)
    
    
    p = figure(y_range=(0, 5), width=1200, height=300, 
               tools=[PanTool(dimensions="width"), 
                      BoxZoomTool(dimensions="width"), 
                      WheelZoomTool(dimensions="width"), 
                      ResetTool(), HoverTool(), UndoTool()],
              )
    legend_item = []
    
    
    mapper1 = linear_cmap(field_name='copyNum', palette=OrRd3+ tuple(reversed(OrRd3)), 
                          low=1 ,high=3)
    mapper2 = linear_cmap(field_name='copyNum', palette=GnBu3 + tuple(reversed(GnBu3)),
                          low=1 ,high=3)
    spinner = Spinner(
        title="Circle size",  # a string to display above the widget
        low=0.5,  # the lowest possible number to pick
        high=10,  # the highest possible number to pick
        step=0.5,  # the increments by which the number can be adjusted
        value=3,  # the initial value to display in the widget
        width=200,  #  the width of the widget in pixels
    )
    
    chrs_int = df['chr_int'].unique()
    chrs = df['chr'].unique()
    end_point = 0
    df['Position_axis'] = end_point
    for c, ch in zip(chrs_int, chrs):
        # ch = f'chr{c}'
        # if  (c == 'X') or (c % 2):
        if type(c) == float and int(c) % 2:
            mapper = mapper1
        elif c == 'X':
            mapper = mapper1
        else:
            mapper = mapper2
            
        # if ch  in df['chr'].unique():
        df.loc[df['chr']==ch, 'Position_axis'] = df.loc[df['chr']==ch, 'Position'] + end_point
        end_point = df['Position_axis'].max()
        data = ColumnDataSource(df.loc[df['chr']==ch])
        pc = p.circle(x='Position_axis', y='copyNum', color=mapper, fill_alpha=0.5, size=3, line_width=0, source=data)#, legend_label=c)
        # legend_item.append((ch, [pc]))
        spinner.js_link("value", pc.glyph, "size")

    # legend = Legend(items=legend_item, location="center", orientation = "horizontal", click_policy="hide")

#     p.x_range.update_from_json({'start':0, 'end':end_point})
#     range_slider = RangeSlider(
#         width=p.width,
#         show_value=False,
#         title="Adjust x-axis range", # a title to display above the slider
#         start=0,  # set the minimum value for the slider
#         end=end_point,  # set the maximum value for the slider
#         step=1,  # increments for the slider
#         value=(0, end_point),  # initial values for slider
#     )
#     range_slider.js_link("value", p.x_range, "start", attr_selector=0)
#     range_slider.js_link("value", p.x_range, "end", attr_selector=1)

    OPTIONS = list(chrs)
    multi_choice = MultiChoice(value=OPTIONS, options=OPTIONS, width=p.width, placeholder='choose a chromosome')
    multi_choice.on_change('value', multichoice_callback)
    multi_choice.js_on_change("value", CustomJS(code="""
        this.toString())
    """))

#     show(multi_choice)
    
    df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')
    data = ColumnDataSource(df_first_pos)
    labels = LabelSet(x='Position_axis', y=4, text='chr',
              x_offset=5, y_offset=10, source=data, render_mode='canvas')
    
    p.toolbar.logo = None
    p.add_layout(labels)
    # p.add_layout(legend, 'above')    
    # plot = layout([[spinner],[range_slider],[p],])
    # plot = layout([[spinner],[p],])
    plot = layout([[spinner],[multi_choice], [p],])
    show(plot)

    
    

In [141]:
def set_plot():
        p = figure(y_range=(0, 5), width=1200, height=300, 
               tools=[PanTool(dimensions="width"), 
                      BoxZoomTool(dimensions="width"), 
                      WheelZoomTool(dimensions="width"), 
                      ResetTool(), HoverTool(), UndoTool()],
              )
        return p
    
def plot_chr(p, df_chr, chrs=None):
    if chrs:
        if type(chrs)==str and chrs in df_chr['chr'].unique():
            df = df_chr.loc[df_chr['chr']==chrs,:].copy()
        if type(chrs)==list and len(chrs)>=1:
            df = df_chr.loc[df_chr['chr'].isin(chrs),:].copy()
    else:
        df = df_chr.copy()
        
    df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce').round(0)
    df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
    df = df.sort_values(by=['chr_int', 'Position'], ascending=True)
    
    mapper1 = linear_cmap(field_name='copyNum', palette=OrRd3+ tuple(reversed(OrRd3)), low=1 ,high=3)
    mapper2 = linear_cmap(field_name='copyNum', palette=GnBu3 + tuple(reversed(GnBu3)),low=1 ,high=3)
    
    chrs_int = df['chr_int'].unique()
    chrs = df['chr'].unique()
    end_point = 0
    df['Position_axis'] = end_point
    for c, ch in zip(chrs_int, chrs):
        print(c,ch)
        if (type(c)==int or type(c)==float) and int(c) % 2:
            mapper = mapper1
        elif c == 'X':
            mapper = mapper1
        else:
            mapper = mapper2
            
        df.loc[df['chr']==ch, 'Position_axis'] = df.loc[df['chr']==ch, 'Position'] + end_point
        end_point = df['Position_axis'].max()
        data = ColumnDataSource(df.loc[df['chr']==ch])
        pc = p.circle(x='Position_axis', y='copyNum', color=mapper, fill_alpha=0.8, size=3, line_width=0, source=data)#, legend_label=c)
    
    df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')
    data = ColumnDataSource(df_first_pos)
    labels = LabelSet(x='Position_axis', y=4, text='chr',x_offset=5, y_offset=10, source=data, render_mode='canvas')
    p.add_layout(labels)
    return p


def multi_choice_wiget(p, lst):
    OPTIONS = list(lst)
    multi_choice = MultiChoice(value=OPTIONS, options=OPTIONS, width=p.width, placeholder='choose a chromosome')
    multi_choice.js_on_change("value", CustomJS(code="""
        console.log('multi_choice: value=' + this.value, 
        this.toString())
    """))
    plot = layout([[multi_choice],[p]])
    
    return plot

    
def multichoice_callback(attr, old, new):
    p = plot_chr(p, df, 'chr1')
#     CustomJS(code="""
#         console.log('multi_choice: test value=' + this.value, 
#         this.toString())
#     """)
#     plot_chr(p, df_chr, chrs=new.split(','))
#     print("Previous label: " + old)
#     print("Updated label: " + new)

In [142]:
import pandas as pd


file = 'test_input.csv'
df = pd.read_csv(file)
p = set_plot()
# p = plot_chr(p, df, 'chr1')
p = multi_choice_wiget(p, ['chr1', 'chr2'])

In [143]:
show(p)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [40]:
df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce')
df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
df = df.sort_values(by=['chr_int', 'Position'], ascending=True)

df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')

In [195]:
df_first_pos

,chr,Position,copyNum,chr_int
0,chr1,4350001,2.165106,1.0
2247,chr2,150001,1.419974,2.0
2988,chr3,450001,1.643338,3.0
3406,chr4,1050001,1.915483,4.0
3803,chr5,1800001,1.849588,5.0
4175,chr6,900001,1.773396,6.0
4533,chr7,1800001,1.758279,7.0
4824,chr8,900001,1.768808,8.0
5123,chr9,450001,1.890266,9.0
446,chr10,600001,1.779802,10.0


In [138]:
df.drop_duplicates(subset='chr', keep='first')

,chr,Position,copyNum
0,chr1,4350001,2.165106
446,chr10,600001,1.779802
706,chr11,3600001,1.523524
961,chr12,300001,1.907495
1233,chr13,20400001,1.891126
1436,chr14,20850001,2.140348
1620,chr15,22950001,2.421781
1776,chr16,3450001,0.998244
1909,chr17,600001,1.682901
2037,chr18,300001,2.041004


In [179]:
import numpy as np
from random import choice
from string import ascii_lowercase

from bokeh.models.tools import *
from bokeh.plotting import *
from bokeh.models import CustomJS



output_notebook()


TOOLS="pan,wheel_zoom,reset,hover,poly_select,box_select"
p = figure(title = "My chart", tools=TOOLS)
p.xaxis.axis_label = 'X'
p.yaxis.axis_label = 'Y'

source = ColumnDataSource(
    data=dict(
        xvals=list(range(0, 10)),
        yvals=list(np.random.normal(0, 1, 10)),
        letters = [choice(ascii_lowercase) for _ in range(10)]
    )
)
p.scatter("xvals", "yvals",source=source,fill_alpha=0.2, size=5)

select_tool = p.select(dict(type=BoxSelectTool))[0]

source.js_on_change('streaming', CustomJS(args=dict(p=p), code="""
        var inds = cb_obj.get('selected')['1d'].indices;
        var d1 = cb_obj.get('data');
        console.log(d1)
        """)
                   )

show(p)

Loading BokehJS ...

In [180]:
source.to_df()

,xvals,yvals,letters
0,0,-0.056900,y
1,1,-1.427325,r
2,2,1.887945,k
3,3,1.129029,f
4,4,-1.203485,p
5,5,1.289698,y
6,6,-0.388431,i
7,7,-0.383802,v
8,8,0.998887,e
9,9,0.471885,d
